In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data11768


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
!unzip data/data11768/guangdong1_round1_train1_20190818.zip -d data/
!unzip data/data11768/guangdong1_round1_testA_20190818.zip -d data/

  inflating: data/guangdong1_round1_testA_20190818/f92f8a70cd500a7f1503432762.jpg  
  inflating: data/guangdong1_round1_testA_20190818/352785352e8262cc1233027626.jpg  
  inflating: data/guangdong1_round1_testA_20190818/bdd09ed39be190dc0831341626.jpg  
  inflating: data/guangdong1_round1_testA_20190818/5b1145ee7b57cd0d1443450612.jpg  
  inflating: data/guangdong1_round1_testA_20190818/ee830b604283225b1029311752.jpg  
  inflating: data/guangdong1_round1_testA_20190818/e92fa127c4e523130852285565.jpg  
  inflating: data/guangdong1_round1_testA_20190818/1b53d2bab298e6af1642297421.jpg  
  inflating: data/guangdong1_round1_testA_20190818/fbd0d3a8683866031217516261.jpg  
  inflating: data/guangdong1_round1_testA_20190818/dcf462f292098e7c1221571123.jpg  
  inflating: data/guangdong1_round1_testA_20190818/59d1b1f56b85f8070934444132.jpg  
  inflating: data/guangdong1_round1_testA_20190818/3d3752af38c033920841000806.jpg  
  inflating: data/guangdong1_round1_testA_20190818/76209c427c7a4409084348050

# 拉取模型

In [34]:
# 如果无法切换可以放弃本地修改：
# !git checkout .
!git checkout develop

activity   deploy		ppdet		  scripts
benchmark  docs			README_cn.md	  setup.py
configs    industrial_tutorial	README_en.md	  test_tipc
dataset    LICENSE		README.md	  tools
demo	   PaddleDetection.zip	requirements.txt
分支 develop 设置为跟踪来自 origin 的远程分支 develop。
切换到一个新分支 'develop'


In [20]:
# Clone PaddleDetection repository
!cd work && git clone https://ghproxy.com/https://github.com/PaddlePaddle/PaddleDetection.git

# Install other dependencies
# !cd PaddleDetection
%cd work/PaddleDetection
!pip install -r requirements.txt

# Compile and install paddledet
!python setup.py install
# 确认测试通过
!python ppdet/modeling/tests/test_architectures.py
%cd ../../

正克隆到 'PaddleDetection'...
remote: Enumerating objects: 256644, done.
remote: Counting objects: 100% (778/778), done.
remote: Compressing objects: 100% (456/456), done.
^C对象中:  11% (28863/256644), 8.96 MiB | 16.00 KiB/s     
[Errno 2] No such file or directory: 'work/PaddleDetection'
/home/aistudio
ERROR: Could not open requirements file: [Errno 2] 没有那个文件或目录: 'requirements.txt'

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
python: can't open file '/home/aistudio/setup.py': [Errno 2] No such file or directory
python: can't open file '/home/aistudio/ppdet/modeling/tests/test_architectures.py': [Errno 2] No such file or directory
/


# 安装cocoAPI

In [5]:
# 安装cocoAPI
!pip install pycocotools

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


# 加载工具库

In [6]:
import os
import json
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import os, sys, zipfile
import urllib.request
import shutil
import random
import numpy as np
from tqdm import tqdm
import cv2
# import kmeans

# 数据集准备
- 参考竞赛页置顶notebook并稍作修改，手动划分10%的图片作为验证集
- 训练集标注文件转换为COCO格式[ 2019广东工业智造创新大赛【赛场一】]
# coco格式转换
将数据集转为飞桨模型库中的coco格式，需要划分验证集
```
data/coco/
├── annotations
│   ├── instances_train2014.json
│   ├── instances_train2017.json
│   ├── instances_val2014.json
│   ├── instances_val2017.json
|   ...
├── train2017
│   ├── 000000000009.jpg
│   ├── 000000580008.jpg
|   ...
├── val2017
│   ├── 000000000139.jpg
│   ├── 000000000285.jpg
|   ...
```

In [7]:
defect_name2label = {
    '破洞': 1, '水渍': 2, '油渍': 2, '污渍': 2, '三丝': 3, '结头': 4, '花板跳': 5, '百脚': 6, '毛粒': 7,
    '粗经': 8, '松经': 9, '断经': 10, '吊经': 11, '粗维': 12, '纬缩': 13, '浆斑': 14, '整经结': 15, '星跳': 16, '跳花': 16,
    '断氨纶': 17, '稀密档': 18, '浪纹档': 18, '色差档': 18, '磨痕': 19, '轧痕': 19, '修痕': 19, '烧毛痕': 19, '死皱': 20, '云织': 20,
    '双纬': 20, '双经': 20, '跳纱': 20, '筘路': 20, '纬纱不良': 20,
}
# defect_name2label = {
#     '破洞': 1, '水渍': 2, '油渍': 3, '污渍': 4, '三丝': 5, '结头': 6, '花板跳': 7, '百脚': 8, '毛粒': 9,
#     '粗经': 10, '松经': 11, '断经': 12, '吊经': 13, '粗维': 14, '纬缩': 15, '浆斑': 16, '整经结': 17, '星跳': 18, '跳花': 19,
#     '断氨纶': 20, '稀密档': 21, '浪纹档': 22, '色差档': 23, '磨痕': 24, '轧痕': 25, '修痕': 26, '烧毛痕': 27, '死皱': 28, '云织': 29,
#     '双纬': 30, '双经': 31, '跳纱': 32, '筘路': 33, '纬纱不良': 34,
# }

# 创建coco格式训练集

In [8]:
class Fabric2COCO:

    def __init__(self,mode="train2017"):
    # def __init__(self,mode="val"):
        self.images = []
        self.annotations = []
        self.categories = []
        self.img_id = 0
        self.ann_id = 0
        self.mode =mode
        # if not os.path.exists("coco/images/{}".format(self.mode)):
        #     os.makedirs("coco/images/{}".format(self.mode))
        if not os.path.exists("PaddleDetection/dataset/coco/{}".format(self.mode)):
            os.makedirs("PaddleDetection/dataset/coco/{}".format(self.mode))

    def to_coco(self, anno_file,img_dir):
        self._init_categories()
        anno_result= pd.read_json(open(anno_file,"r"))
        anno_result = anno_result.head(int(anno_result['name'].count()*0.9))
        # anno_result = anno_result.tail(int(anno_result['name'].count()*0.1)+1)        
        name_list=anno_result["name"].unique()
        for img_name in name_list:
            img_anno = anno_result[anno_result["name"] == img_name]
            bboxs = img_anno["bbox"].tolist()
            defect_names = img_anno["defect_name"].tolist()
            assert img_anno["name"].unique()[0] == img_name

            img_path=os.path.join(img_dir,img_name)
            # img =cv2.imread(img_path)
            # h,w,c=img.shape
            h,w=1000,2446
            self.images.append(self._image(img_path,h, w))

            self._cp_img(img_path)

            for bbox, defect_name in zip(bboxs, defect_names):
                label= defect_name2label[defect_name]
                annotation = self._annotation(label, bbox)
                self.annotations.append(annotation)
                self.ann_id += 1
            self.img_id += 1
        instance = {}
        instance['info'] = 'fabric defect'
        instance['license'] = ['none']
        instance['images'] = self.images
        instance['annotations'] = self.annotations
        instance['categories'] = self.categories
        return instance

    def _init_categories(self):
        for v in range(1,21):
            print(v)
            category = {}
            category['id'] = v
            category['name'] = str(v)
            category['supercategory'] = 'defect_name'
            self.categories.append(category)
        # for k, v in defect_name2label.items():
        #     category = {}
        #     category['id'] = v
        #     category['name'] = k
        #     category['supercategory'] = 'defect_name'
        #     self.categories.append(category)

    def _image(self, path,h,w):
        image = {}
        image['height'] = h
        image['width'] = w
        image['id'] = self.img_id
        image['file_name'] = os.path.basename(path)
        return image

    def _annotation(self,label,bbox):
        area=(bbox[2]-bbox[0])*(bbox[3]-bbox[1])
        points=[[bbox[0],bbox[1]],[bbox[2],bbox[1]],[bbox[2],bbox[3]],[bbox[0],bbox[3]]]
        annotation = {}
        annotation['id'] = self.ann_id
        annotation['image_id'] = self.img_id
        annotation['category_id'] = label
        annotation['segmentation'] = [np.asarray(points).flatten().tolist()]
        annotation['bbox'] = self._get_box(points)
        annotation['iscrowd'] = 0
        annotation['area'] = area
        return annotation

    def _cp_img(self, img_path):
        shutil.copy(img_path, os.path.join("PaddleDetection/dataset/coco/{}".format(self.mode), os.path.basename(img_path)))
        # shutil.copy(img_path, os.path.join("coco/images/{}".format(self.mode), os.path.basename(img_path)))
    def _get_box(self, points):
        min_x = min_y = np.inf
        max_x = max_y = 0
        for x, y in points:
            min_x = min(min_x, x)
            min_y = min(min_y, y)
            max_x = max(max_x, x)
            max_y = max(max_y, y)
        '''coco,[x,y,w,h]'''
        return [min_x, min_y, max_x - min_x, max_y - min_y]
    def save_coco_json(self, instance, save_path):
        import json
        with open(save_path, 'w') as fp:
            json.dump(instance, fp, indent=1, separators=(',', ': '))

In [9]:
'''转换有瑕疵的样本为coco格式'''
img_dir = "data/guangdong1_round1_train1_20190818/defect_Images"
anno_dir="data/guangdong1_round1_train1_20190818/Annotations/anno_train.json"
fabric2coco = Fabric2COCO()
train_instance = fabric2coco.to_coco(anno_dir,img_dir)
if not os.path.exists("PaddleDetection/dataset/coco/annotations/"):
    os.makedirs("PaddleDetection/dataset/coco/annotations/")
fabric2coco.save_coco_json(train_instance, "PaddleDetection/dataset/coco/annotations/"+'instances_{}.json'.format("train2017"))
# fabric2coco.save_coco_json(train_instance, "coco/annotations/"+'instances_{}.json'.format("val"))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


# 创建coco格式验证集

In [10]:
class Fabric2COCO:

    def __init__(self,mode="val2017"):
        self.images = []
        self.annotations = []
        self.categories = []
        self.img_id = 0
        self.ann_id = 0
        self.mode =mode
        # if not os.path.exists("coco/images/{}".format(self.mode)):
        #     os.makedirs("coco/images/{}".format(self.mode))
        if not os.path.exists("PaddleDetection/dataset/coco/{}".format(self.mode)):
            os.makedirs("PaddleDetection/dataset/coco/{}".format(self.mode))

    def to_coco(self, anno_file,img_dir):
        self._init_categories()
        anno_result= pd.read_json(open(anno_file,"r"))
        anno_result = anno_result.tail(int(anno_result['name'].count()*0.1)+1)        
        name_list=anno_result["name"].unique()
        for img_name in name_list:
            img_anno = anno_result[anno_result["name"] == img_name]
            bboxs = img_anno["bbox"].tolist()
            defect_names = img_anno["defect_name"].tolist()
            assert img_anno["name"].unique()[0] == img_name

            img_path=os.path.join(img_dir,img_name)
            # img =cv2.imread(img_path)
            # h,w,c=img.shape
            h,w=1000,2446
            self.images.append(self._image(img_path,h, w))

            self._cp_img(img_path)

            for bbox, defect_name in zip(bboxs, defect_names):
                label= defect_name2label[defect_name]
                annotation = self._annotation(label, bbox)
                self.annotations.append(annotation)
                self.ann_id += 1
            self.img_id += 1
        instance = {}
        instance['info'] = 'fabric defect'
        instance['license'] = ['none']
        instance['images'] = self.images
        instance['annotations'] = self.annotations
        instance['categories'] = self.categories
        return instance

    def _init_categories(self):
        for v in range(1,21):
            print(v)
            category = {}
            category['id'] = v
            category['name'] = str(v)
            category['supercategory'] = 'defect_name'
            self.categories.append(category)
        # for k, v in defect_name2label.items():
        #     category = {}
        #     category['id'] = v
        #     category['name'] = k
        #     category['supercategory'] = 'defect_name'
        #     self.categories.append(category)

    def _image(self, path,h,w):
        image = {}
        image['height'] = h
        image['width'] = w
        image['id'] = self.img_id
        image['file_name'] = os.path.basename(path)
        return image

    def _annotation(self,label,bbox):
        area=(bbox[2]-bbox[0])*(bbox[3]-bbox[1])
        points=[[bbox[0],bbox[1]],[bbox[2],bbox[1]],[bbox[2],bbox[3]],[bbox[0],bbox[3]]]
        annotation = {}
        annotation['id'] = self.ann_id
        annotation['image_id'] = self.img_id
        annotation['category_id'] = label
        annotation['segmentation'] = [np.asarray(points).flatten().tolist()]
        annotation['bbox'] = self._get_box(points)
        annotation['iscrowd'] = 0
        annotation['area'] = area
        return annotation

    def _cp_img(self, img_path):
        shutil.copy(img_path, os.path.join("PaddleDetection/dataset/coco/{}".format(self.mode), os.path.basename(img_path)))
        # shutil.copy(img_path, os.path.join("coco/images/{}".format(self.mode), os.path.basename(img_path)))
    def _get_box(self, points):
        min_x = min_y = np.inf
        max_x = max_y = 0
        for x, y in points:
            min_x = min(min_x, x)
            min_y = min(min_y, y)
            max_x = max(max_x, x)
            max_y = max(max_y, y)
        '''coco,[x,y,w,h]'''
        return [min_x, min_y, max_x - min_x, max_y - min_y]
    def save_coco_json(self, instance, save_path):
        import json
        with open(save_path, 'w') as fp:
            json.dump(instance, fp, indent=1, separators=(',', ': '))

In [11]:
'''转换有瑕疵的样本为coco格式'''
img_dir = "data/guangdong1_round1_train1_20190818/defect_Images"
anno_dir="data/guangdong1_round1_train1_20190818/Annotations/anno_train.json"
fabric2coco = Fabric2COCO()
train_instance = fabric2coco.to_coco(anno_dir,img_dir)
if not os.path.exists("PaddleDetection/dataset/coco/annotations/"):
    os.makedirs("PaddleDetection/dataset/coco/annotations/")
fabric2coco.save_coco_json(train_instance, "PaddleDetection/dataset/coco/annotations/"+'instances_{}.json'.format("val2017"))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


# 整理后最终文件目录格式
```
./coco/
├── annotations
│   ├── instances_train2014.json
│   ├── instances_train2017.json
|   ...
├── train2017
│   ├── 5cac654fb8e699da1546312852.jpg
│   ├── 14d68415c6f020021536006312.jpg
|   ...
├── val2017
│   ├── 6cf97c00f1a180120939190444.jpg
│   ├── cb42508af46ed0a50817439434.jpg
|   ...
```